8-puzzle problem

In [1]:
import random
import copy

In [2]:
# Generating solvable state
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
n_problems = 100
n_iters = 50
moves = [[-1, 0, 'Up'], [1, 0, 'Down'], [0, 1, 'Right'], [0, -1, 'Left']]
all_problems = []
rows, cols = len(goal_state), len(goal_state[0])

for _ in range(n_problems):
    new_state = copy.deepcopy(goal_state)
    # Get the position of 0
    empty_cell = 0
    for row_index, row in enumerate(new_state):
        if empty_cell in row:
            col_index = row.index(empty_cell)
            empty_r, empty_c = row_index, col_index

    
    # Shuffle
    for _ in range(n_iters): 
        dx, dy, _ = moves[random.randint(0, 3)] # 0 and 3 included
        x, y = empty_r + dx, empty_c + dy
        if 0 <= x < rows and 0 <= y < cols:
            new_state[empty_r][empty_c], new_state[x][y] =\
            new_state[x][y], new_state[empty_r][empty_c]
            empty_r, empty_c = x, y
        
    all_problems.append(copy.deepcopy(new_state))


In [3]:
# n_iters = 10000
# all_problems = []
# new_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
# goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# # Get the position of 0
# empty_cell = 0
# for row_index, row in enumerate(new_state):
#     if empty_cell in row:
#         col_index = row.index(empty_cell)
#         empty_r, empty_c = row_index, col_index

# for i in range(n_iters):
#     rand = random.randint(0, 3)
#     dx, dy, _ = moves[rand] # 0 and 3 included
#     x, y = empty_r + dx, empty_c + dy
#     if 0 <= x < rows and 0 <= y < cols:
#         new_state[empty_r][empty_c], new_state[x][y] =\
#         new_state[x][y], new_state[empty_r][empty_c]
#         empty_r, empty_c = x, y

#     if i % 10 == 0:
#         all_problems.append(copy.deepcopy(new_state))
        
# print(all_problems)

In [4]:
# Define heuristic function (Manhataen Distance is used)
def manhattan_dist(current_state, goal_state):
    """
    Args:
        current_state (matrix) : the matrix to be checked
        goal_state (matrix) : the goal matrix
    Returns:
        distance (int) : the manhattan distance between current state and goal state
    """
    distance = 0
    for row in range(len(current_state)):
        for col in range(len(current_state[0])):
            state = current_state[row][col]
            # Finding the position of state in the goal_state
            for i in range(len(goal_state)):
                if state in goal_state[i]:
                    goal_col = goal_state[i].index(state)
                    goal_row = i
            dist = abs(row - goal_row) + abs(col - goal_col)
            distance += dist

    return distance


In [5]:
# Get all possible neighbor states (states after 1 valid move)
def get_neighbors(current_state):
    """
    Args:
        current_state (narray(n, n)) : Matrix of the current state
    Returns:
        possible_next_states (List of matrix) : List of all possible next states
    """
    # Find the empty cell (cell with 0)
    empty = 0
    total_rows = len(current_state)
    total_cols = len(current_state[0])
    for i in range(total_rows):
        if empty in current_state[i]:
            empty_col = current_state[i].index(empty)
            empty_row = i

    # Define possible moves
    possible_next_states = []
    valid_moves = [[-1, 0, 'Up'], [1, 0, 'Down'], [0, 1, 'Right'], [0, -1, 'Left']]
    for dx, dy, _ in valid_moves:
        new_row = empty_row + dx
        new_col = empty_col + dy
        if 0 <= new_row < total_rows and 0 <= new_col < total_cols:
            decoy = copy.deepcopy(current_state)
            decoy[empty_row][empty_col], decoy[new_row][new_col] =\
            decoy[new_row][new_col], decoy[empty_row][empty_col]
            possible_next_states.append(decoy)
    
    return possible_next_states   

In [6]:
def hill_climbing_search(initial_state):
    """
    Main function for the search (hill climbing algorithm)
    Args:
        initial_state (narray(n, n)) : Matrix of the initial state
    Returns:
        local_optimum (narray(n, n)) : Local maximum 
    """
    current_state = initial_state
    while True:
        best_neighbor, best_dist = None, float('inf')
        for neighbor in get_neighbors(current_state):
            if manhattan_dist(neighbor, goal_state) < best_dist:
                best_neighbor, best_dist = neighbor, manhattan_dist(neighbor, goal_state)

        current_dist = manhattan_dist(current_state, goal_state)
        if current_dist > best_dist:
            current_state = best_neighbor
        else:
            return current_state
            

In [7]:
# Solve the puzzles using hill climbing search
print("Problem\t\t\t\t    Initial Dist\tOptimal Solution\t\tFinal Dist")
print("-----------------------------------------------------------------------------------------------")
for problem in all_problems:
    solution = hill_climbing_search(problem)
    initial_dist = manhattan_dist(problem, goal_state)
    final_dist = manhattan_dist(solution, goal_state)
    print(f"{problem}\t{initial_dist}\t{solution}\t{final_dist}")

Problem				    Initial Dist	Optimal Solution		Final Dist
-----------------------------------------------------------------------------------------------
[[0, 2, 3], [1, 7, 5], [8, 4, 6]]	12	[[1, 2, 3], [7, 4, 5], [8, 0, 6]]	6
[[1, 5, 0], [4, 3, 2], [7, 8, 6]]	8	[[1, 5, 2], [4, 3, 6], [7, 8, 0]]	4
[[1, 4, 2], [7, 3, 6], [0, 5, 8]]	10	[[1, 4, 2], [7, 3, 6], [0, 5, 8]]	10
[[1, 0, 3], [5, 2, 6], [4, 7, 8]]	8	[[1, 2, 3], [5, 0, 6], [4, 7, 8]]	6
[[1, 2, 3], [4, 8, 5], [7, 0, 6]]	4	[[1, 2, 3], [4, 8, 5], [7, 0, 6]]	4
[[1, 2, 3], [7, 8, 5], [6, 4, 0]]	8	[[1, 2, 3], [7, 8, 5], [6, 4, 0]]	8
[[1, 3, 6], [5, 0, 2], [4, 7, 8]]	10	[[1, 3, 6], [5, 2, 0], [4, 7, 8]]	8
[[1, 2, 3], [4, 5, 6], [0, 7, 8]]	4	[[1, 2, 3], [4, 5, 6], [7, 8, 0]]	0
[[1, 3, 6], [4, 2, 8], [0, 7, 5]]	10	[[1, 3, 6], [4, 2, 8], [7, 5, 0]]	6
[[1, 0, 2], [4, 6, 3], [7, 5, 8]]	8	[[1, 2, 3], [4, 6, 0], [7, 5, 8]]	4
[[5, 6, 3], [0, 2, 8], [1, 4, 7]]	16	[[5, 6, 3], [1, 2, 8], [4, 7, 0]]	10
[[7, 1, 2], [8, 4, 0], [5, 3, 6]]	14	[[7, 1, 2],

In [8]:
def stochastic_hill_climbing(initial_state):
    """ 
    Main function for stochastic hill climbing search algorithm
    Args:
        initial_state (narray(n, n)) : Initial matrix of the problem
    Returns:
        current_state (narray(n, n)) : Final matrix with the optimal solution
    """
    current = initial_state
    current_dist = manhattan_dist(current, goal_state)

    while True:
        best_neighbors = []
        for neighbor in get_neighbors(current):
            neighbor_dist = manhattan_dist(neighbor, goal_state)
            if neighbor_dist < current_dist:
                best_neighbors.append(neighbor)

        if len(best_neighbors) == 0:
            return current
        
        rand_idx = random.randint(0, len(best_neighbors) - 1)
        current = best_neighbors[rand_idx]
        current_dist = manhattan_dist(current, goal_state)

In [9]:
from tabulate import tabulate

headers = ["Problem", "Initial Dist", "Stochastic Soln", "Final Dist"]
data = []
for problem in all_problems:
    soln = stochastic_hill_climbing(problem)
    soln_dist = manhattan_dist(soln, goal_state)
    init_dist = manhattan_dist(problem, goal_state)
    data.append([problem, init_dist, soln, soln_dist])

print(tabulate(data, headers= headers))
    

Problem                              Initial Dist  Stochastic Soln                      Final Dist
---------------------------------  --------------  ---------------------------------  ------------
[[0, 2, 3], [1, 7, 5], [8, 4, 6]]              12  [[1, 2, 3], [7, 4, 5], [8, 0, 6]]             6
[[1, 5, 0], [4, 3, 2], [7, 8, 6]]               8  [[1, 5, 2], [4, 3, 6], [7, 8, 0]]             4
[[1, 4, 2], [7, 3, 6], [0, 5, 8]]              10  [[1, 4, 2], [7, 3, 6], [0, 5, 8]]            10
[[1, 0, 3], [5, 2, 6], [4, 7, 8]]               8  [[1, 2, 3], [5, 0, 6], [4, 7, 8]]             6
[[1, 2, 3], [4, 8, 5], [7, 0, 6]]               4  [[1, 2, 3], [4, 8, 5], [7, 0, 6]]             4
[[1, 2, 3], [7, 8, 5], [6, 4, 0]]               8  [[1, 2, 3], [7, 8, 5], [6, 4, 0]]             8
[[1, 3, 6], [5, 0, 2], [4, 7, 8]]              10  [[1, 3, 6], [5, 2, 0], [4, 7, 8]]             8
[[1, 2, 3], [4, 5, 6], [0, 7, 8]]               4  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]             0
[[1, 3, 6]

In [10]:
# Simulated annueling

In [11]:
def cooling_schedule(T0, cooling_rate, k):
    """Calculates the temperature T at k-th iteration
    Args:
        T0 (float) : Initial temperature
        cooling_rate (float) : Cooling factor
        k (int) : Iteration until current state
    Returns:
        Tf (int) : Final temperature
    """
    Tf = cooling_rate ** k * T0
    return Tf


In [12]:
import math

In [13]:
def simulated_annealing(problem, T0, cooling_rate):
    """Main function for simulated annealing: decreasing prob of selecting worse decision
    Args:
        problem (narray(n, n)) : Initial state matrix
        T0 (float) : Initial temperature
        cooling_rate (float) : Cooling factor
    Returns:
        current_state (narray(n, n)) : Final optimal solution
    """
    threshold = 0.1
    current = problem
    iter = 1

    while T0 > threshold:
        in_iteration = current
        for neighbor in get_neighbors(current):
            delE = manhattan_dist(neighbor, goal_state) - manhattan_dist(current, goal_state)
            if delE < 0:
                in_iteration = neighbor
                break
            else:
                prob = math.e ** (-delE/T0)
                rand = random.random() # doesnot take any argument
                if rand < prob:
                    in_iteration = neighbor
                    break

        if in_iteration is current:
            return current
        current = in_iteration
        T0 = cooling_schedule(T0, cooling_rate, iter)
        iter += 1

    return current


In [14]:
headers = ["Problem", "Initial Dist", "Simulation Ann. Soln", "Final Dist"]
data = []
for problem in all_problems:
    soln = simulated_annealing(problem, T0= 0.95, cooling_rate= 0.8)
    soln_dist = manhattan_dist(soln, goal_state)
    init_dist = manhattan_dist(problem, goal_state)
    data.append([problem, init_dist, soln, soln_dist])

print(tabulate(data, headers= headers))

Problem                              Initial Dist  Simulation Ann. Soln                 Final Dist
---------------------------------  --------------  ---------------------------------  ------------
[[0, 2, 3], [1, 7, 5], [8, 4, 6]]              12  [[1, 2, 3], [0, 7, 5], [8, 4, 6]]            10
[[1, 5, 0], [4, 3, 2], [7, 8, 6]]               8  [[1, 5, 2], [4, 3, 6], [7, 8, 0]]             4
[[1, 4, 2], [7, 3, 6], [0, 5, 8]]              10  [[1, 4, 2], [0, 3, 6], [7, 5, 8]]            10
[[1, 0, 3], [5, 2, 6], [4, 7, 8]]               8  [[1, 2, 3], [5, 0, 6], [4, 7, 8]]             6
[[1, 2, 3], [4, 8, 5], [7, 0, 6]]               4  [[1, 2, 3], [4, 0, 5], [7, 8, 6]]             4
[[1, 2, 3], [7, 8, 5], [6, 4, 0]]               8  [[1, 2, 3], [7, 8, 5], [6, 4, 0]]             8
[[1, 3, 6], [5, 0, 2], [4, 7, 8]]              10  [[1, 3, 6], [5, 7, 2], [4, 0, 8]]            10
[[1, 2, 3], [4, 5, 6], [0, 7, 8]]               4  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]             0
[[1, 3, 6]

In [15]:
# Optimal solution A* Search
# Datastructure :
#   PriorityQueue (for not-yet expanded nodes)
#   Set (for expanded nodes) Caution: the objects in the set must be hashable (immutable)



In [ ]:
class Node(object):
# Contains
#   state : Current state
#   g(n) : Cost to reach the current state
#   h(n) : Heurstic (like manhattan distance) to reach the goal state
#   f(n) : Total cost f(n) = g(n) + h(n)
#   Parent node : A reference to parent node to reconstruct the path if goal is found    (We didn't implement it)
    def __init__(self, state, gn= 0, hn= manhattan_dist, parent= None):
        self.state = state
        self.gn = gn
        self.hn = hn(state, goal_state)
        self.fn = self.gn + self.hn
        self.parent = parent

    def is_goal(self):
        if self.state == goal_state:
            return True
        return False
    
    # Overload '>' operator
    def __lt__(self, other):
        return self.fn < other.fn

In [17]:
import heapq
# IMPORTANT
# I want to write a pathway of the solution 

# A_star
def A_star_search(problem, goal_state):
    """
    Optimal solution 
    Args:
        problem (narray (n, n)) : Initial array of the problem
        goal_state (narray (n, n)) : Final state we want to achieve
    Returns:
        current (narray (n, n)) : The optimal solution achieved by the A*
    """
    queue = []
    hash = set()
    start_node = Node(problem)
    heapq.heappush(queue, start_node)
    
    # Main loop
    while queue:
        current_node = heapq.heappop(queue)
        if current_node.is_goal():
            # Return the path from initial to current
            return current_node.state
        hash.add(current_node)
        neighbor_states= get_neighbors(current_node.state)
        for neighbor_state in neighbor_states:
            nbr = Node(neighbor_state, gn= current_node.gn + 1, parent= current_node)
            if nbr in hash:
                continue
            if nbr in queue:
                idx = queue.index(nbr)
                if nbr.gn < queue[idx].gn:
                    queue.pop(idx)
                    heapq.heappush(queue, nbr)
                continue
            heapq.heappush(queue, nbr)
        
    return current_node.state

In [19]:
headers = ["Problem", "Initial Dist", "A* Soln", "Final Dist"]
data = []
for problem in all_problems[:10]:
    soln = A_star_search(problem, goal_state)
    soln_dist = manhattan_dist(soln, goal_state)
    init_dist = manhattan_dist(problem, goal_state)
    data.append([problem, init_dist, soln, soln_dist])

print(tabulate(data, headers= headers))

Problem                              Initial Dist  A* Soln                              Final Dist
---------------------------------  --------------  ---------------------------------  ------------
[[0, 2, 3], [1, 7, 5], [8, 4, 6]]              12  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]             0
[[1, 5, 0], [4, 3, 2], [7, 8, 6]]               8  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]             0
[[1, 4, 2], [7, 3, 6], [0, 5, 8]]              10  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]             0
[[1, 0, 3], [5, 2, 6], [4, 7, 8]]               8  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]             0
[[1, 2, 3], [4, 8, 5], [7, 0, 6]]               4  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]             0
[[1, 2, 3], [7, 8, 5], [6, 4, 0]]               8  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]             0
[[1, 3, 6], [5, 0, 2], [4, 7, 8]]              10  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]             0
[[1, 2, 3], [4, 5, 6], [0, 7, 8]]               4  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]             0
[[1, 3, 6]

In [ ]:
# Graph for
# Success rate
# Search cost

8 Queen's problem

In [20]:
# Initial state

# Successor states
# 1 for each column and 7 states per column

In [31]:
# Heuristic function
def attacking_queens(state):
    """
    Returns the number of queens attacking eachother.
    This is the heurstic function used.
    Args:
        state (list) : representation of the position of the 8 queens
    Return:
        attack (int) : total number of attacks
    """
    attack = 0
    cols = len(state)
    rows = cols

    for i in range(cols - 1):
        for j in range(i+1, cols):
            state_i = state[i]
            state_j = state[j]
            if state_i == state_j or j - i == abs(state_j - state_i): # Assuming same row and diagonal are mutually exclusive
                attack += 1

    return attack

In [32]:
def get_neighbors_queen(state):
    """Generates all possible states from the input state
    Args:
        state (list) : Initial state
    Returns:
        neigbors (list) : List of all possible neighbor states
    """
    # total columns and total rows = 8
    neighbors = []
    for  col in range(8):
        r, c = state[col], col # position of queen on column 'col' in the initial state
        for row in range(8):
            if row == r: # the queen was already on that row
                continue 
            neighbor = copy.deepcopy(state)
            neighbor[col] = row
            neighbors.append(neighbor)

    return neighbors

In [33]:
def hill_climbing_for_queen(problem):
    """Solving 8-queen problem using hill-climbing algorithm
    Args:
        problem (list) : List representing row of a queen problem[i] in each column i
    Returns:
        solution (list) : A local optima solution found
    """

    best = problem
    minimum = attacking_queens(best)
    while True: # Until local optima is reached
        neighbors = get_neighbors_queen(best) # Deep copy will be made
        cost_best = float('inf')
        neighbor_best = None
        for nbr in neighbors:
            cost_nbr = attacking_queens(nbr)
            if cost_nbr < cost_best:
                cost_best = cost_nbr
                neighbor_best = nbr

        if cost_best >=  minimum:
            return best
        
        best = neighbor_best
        minimum = cost_best


In [35]:
# Randomly generate 50 problems
problem_queens = []
sample = [0, 1, 2, 3, 4, 5, 6, 7]
for i in range(50):
    pbl = copy.deepcopy(sample)
    j = random.randint(0, 7)
    k = random.randint(0, 7)
    pbl[j], pbl[k] = pbl[k], pbl[j]
    problem_queens.append(pbl)

# 
headers = ["Problem", "No of Attacking Queens", "Hill-Climbing Soln", "Final Attacking Queen"]
data = []
for problem in problem_queens:
    init_attack = attacking_queens(problem)
    soln = hill_climbing_for_queen(problem)
    soln_attack = attacking_queens(soln)
    data.append([problem, init_attack, soln, soln_attack])

print(tabulate(data, headers= headers))

Problem                     No of Attacking Queens  Hill-Climbing Soln          Final Attacking Queen
------------------------  ------------------------  ------------------------  -----------------------
[7, 1, 2, 3, 4, 5, 6, 0]                        16  [7, 0, 4, 1, 1, 5, 6, 2]                        2
[4, 1, 2, 3, 0, 5, 6, 7]                        18  [4, 6, 1, 5, 2, 0, 3, 7]                        0
[0, 7, 2, 3, 4, 5, 6, 1]                        18  [3, 7, 0, 4, 1, 5, 6, 2]                        1
[0, 1, 2, 3, 6, 5, 4, 7]                        18  [1, 5, 2, 6, 3, 0, 4, 7]                        1
[0, 1, 2, 7, 4, 5, 6, 3]                        18  [1, 5, 7, 2, 4, 0, 6, 3]                        2
[0, 1, 2, 3, 7, 5, 6, 4]                        16  [1, 6, 0, 3, 7, 5, 2, 4]                        2
[0, 1, 4, 3, 2, 5, 6, 7]                        18  [1, 3, 5, 0, 2, 4, 6, 7]                        1
[0, 1, 2, 3, 6, 5, 4, 7]                        18  [1, 5, 2, 6, 3, 0, 4, 7]      